# Instalacion de librerias adicionales

In [1]:
! pip install opendatasets --user

# Importar Librerias

In [2]:
import opendatasets as od
import pandas as pd
import requests

# Descargar el dataset

In [3]:
dataset_url='https://www.kaggle.com/datasets/juanmartnrodrguez/argentina-dataset-propiedades'
od.download(dataset_url)

100%|██████████| 255M/255M [00:39<00:00, 6.69MB/s] 


# Introduccion

**Contexto Empresarial:** Eres un cientifico de datos de una organizacion. Dicha organizacion se dedica al desarrollo inmobiliario a lo largo de Argentina. El modelo de negocio es comprar terrenos, contratar a una constructora, desarrollar y vender o alquilar propiedades. En los ultimos años la empresa no ha tenido exito en las ventas y en los lugares a desarrollar. La gerencia plantea la hipotesis de que localidad o provincia tiene mayor rentabilidad para desarrollar negocios inmobililarios.

**Problema Empresarial:** El equipo de ventas junto con la gerencia se acerca hacia nosotros para responder la siguiente pregunta: *¿Cual es la mejor zona para desarrollar propiedades que se puedan vender rapido?, si es asi, ¿Que tipo de propiedades se suelen vender mas rapido?*

**Contexto Analitico:** El departamento de ventas nos ha provisto una base de datos con todas las ventas de propiedades en Argentina que contiene atributos como fecha de inicio de publicacion de la propiedad, fecha de fin de la publicacion de la propiedad, precio de venta,cantidad de ambientes, baños, superficie total y cubierta, y mas. Veremos que con una regresion lineal podemos cuantificar la correlacion entre la variable dependiente (tipo de propiedad) y las variables dependientes (Precio, tiempo en el mercado, cantidad de baños, habitaciones,etc.)

El caso esta estructurado de la siguiente manera: (1) realizaremos un analisis de datos explotaratorio para investigar visualmente rangos de precio para distintos tipos de propiedades, cantidad de propiedades vendidas por provincias y diferencias entre localidades. (2) utilizar los conocimientos sobre regresion lineal para ajustar el modelo y finalmente (3) abordar el tema principal de la hipotesis.

# Exploracion de datos

In [15]:
df = pd.read_csv('./argentina-dataset-propiedades/ar_properties.csv')
df.head()

,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,...,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
0,wdQ5hWhv8P14T7Sh9g4QCg==,Propiedad,2020-12-25,9999-12-31,2020-12-25,-32.716652,-68.642692,Argentina,Mendoza,NaN,...,NaN,350.0,350.0,NaN,NaN,NaN,Excelentes Lotes Sobre Ruta 34,Corredor Responsable: VICTOR E. MONTIVERO - C....,Lote,Venta
1,nnMBYZ4RMRY+vm753EtA+g==,Propiedad,2020-12-25,9999-12-31,2020-12-25,-24.797723,-65.467514,Argentina,Salta,NaN,...,NaN,1541.0,1541.0,NaN,NaN,Mensual,TERRENO + VENTA + JARDINES DE SAN LORENZO +150...,Corredor Responsable: Pablo Castañeda - C.U.C....,Lote,Venta
2,+dnVA1K6JxzL1zAjOEQ1pA==,Propiedad,2020-12-25,2020-12-29,2020-12-25,-34.919373,-58.020591,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,...,NaN,1000.0,1000.0,NaN,NaN,Mensual,Lote en Venta de 1000 m2 en La Plata,Corredor Responsable: Rico Sebastián - Martill...,Lote,Venta
3,dLHXKN5/sRZpm9Yk0yI2nA==,Propiedad,2020-12-25,2020-12-29,2020-12-25,-34.919455,-58.024807,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,...,NaN,1000.0,1000.0,NaN,NaN,Mensual,Lote en Venta de 1000 m2 en La Plata,Corredor Responsable: Rico Sebastián - Martill...,Lote,Venta
4,wtw/k887EPipd37UYHKb1Q==,Propiedad,2020-12-25,9999-12-31,2020-12-25,-34.364924,-58.783143,Argentina,Bs.As. G.B.A. Zona Norte,Escobar,...,NaN,18164.0,18164.0,NaN,NaN,Mensual,PANAMERICANA 47300,Nave principal 66 x 90 m: 6005 m2 cubiertos...,Otro,Venta


In [6]:
df.shape

(1000000, 25)

In [12]:
df.columns

Index(['id', 'ad_type', 'start_date', 'end_date', 'created_on', 'lat', 'lon',
       'l1', 'l2', 'l3', 'l4', 'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms',
       'surface_total', 'surface_covered', 'price', 'currency', 'price_period',
       'title', 'description', 'property_type', 'operation_type'],
      dtype='object')

Las variables disponibles son:

- *type* - Tipo de aviso (Propiedad, Desarrollo/Proyecto).    
- *id* - Identificador del aviso. No es único: si el aviso es actualizado por la inmobiliaria (nueva versión del aviso) se crea un nuevo registro con la misma id pero distintas fechas: de alta y de baja.
- *start_date* - Fecha de alta del aviso.
- *end_date* - Fecha de baja del aviso.
- *created_on* - Fecha de alta de la primera versión del aviso.
- *place* - Campos referidos a la ubicación de la propiedad o del desarrollo.
    * lat - Latitud.
    * lon - Longitud.
    * l1 - Nivel administrativo 1: país.
    * l2 - Nivel administrativo 2: usualmente provincia.
    * l3 - Nivel administrativo 3: usualmente ciudad.
    * l4 - Nivel administrativo 4: usualmente barrio.
- *property* - Campos relativos a la propiedad (vacío si el aviso es de un desarrollo/proyecto).
    * operation - Tipo de operación (Venta, Alquiler).
    * type - Tipo de propiedad (Casa, Departamento, PH).
    * rooms - Cantidad de ambientes (útil en Argentina).
    * bedrooms - Cantidad de dormitorios (útil en el resto de los países).
    * bathrooms - Cantidad de baños.
    * surface_total - Superficie total en m².
    * surface_covered - Superficie cubierta en m².
    * price - Precio publicado en el anuncio.
    * currency - Moneda del precio publicado.
    * price_period - Periodo del precio (Diario, Semanal, Mensual)
    * title - Título del anuncio.
    * description - Descripción del anuncio.
- *name* - Nombre del desarrollo.
- *description* - Descripción del anuncio.


Primero filtrar por las caracteristicas que precisamos:
- Que sea de Argentina
- Que la propiedad este terminada y vendida
- Que sea una propiedad; que no sea Desarrollo o proyecto
- Que en el precio no este en nulo y que esté en dolares


In [19]:
df = df[(df['l1'] == 'Argentina') & (df['currency'] == 'USD') & (df['operation_type'] == 'Venta')]
df

,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,...,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
64,cTvfaO6v0/zhzA7gh78rKA==,Propiedad,2020-12-25,9999-12-31,2020-12-25,NaN,NaN,Argentina,Bs.As. G.B.A. Zona Norte,José C Paz,...,3.0,NaN,280.0,280000.0,USD,NaN,PASO DE LOS PATOS 3900,Solida casa de excelente calidad constructiva...,Casa,Venta
65,zJ23nVc7AbbBBDoEsOS9Gg==,Propiedad,2020-12-25,9999-12-31,2020-12-25,NaN,NaN,Argentina,Santa Fe,Alvear,...,NaN,NaN,NaN,38000.0,USD,NaN,OPORTUNIDAD UNICA EN ALVEAR - LOTE- IDEAL COME...,<b>OPORTUNIDAD UNICA EN ALVEAR - LOTE- IDEAL C...,Lote,Venta
66,onxzpKWT8/CsuyIlMKiDwQ==,Propiedad,2020-12-25,2021-01-03,2020-12-25,-40.123493,-71.295111,Argentina,Neuquén,Lácar,...,NaN,NaN,NaN,80000.0,USD,Mensual,Terreno - San Martin De Los Andes,Terreno con Gas Natural en Callejòn de Gingins...,Lote,Venta
67,IvEaH6gFGvTej4BAx+Qw1Q==,Propiedad,2020-12-25,2020-12-29,2020-12-25,-34.634550,-58.618627,Argentina,Bs.As. G.B.A. Zona Oeste,Morón,...,NaN,336.0,197.0,126000.0,USD,NaN,VENTA TERRENO ZONA INDUSTRIAL 335 M2 HAEDO NORTE,Corredor Responsable: Jorge Roca - CMCPSM 2550...,Lote,Venta
68,lE15JP9i/uFdRC/UYkI1eA==,Propiedad,2020-12-25,2021-01-10,2020-12-25,-32.465743,-60.844131,Argentina,Santa Fe,Iriondo,...,NaN,790.0,790.0,23500.0,USD,NaN,Terreno en Venta Solares Norte Oliveros,Corredor Responsable: Sebastian Pellegrini - C...,Lote,Venta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999977,3G/cvrya9eXZhMNPlIHwwQ==,Propiedad,2020-05-12,9999-12-31,2020-05-12,-34.863213,-58.392490,Argentina,Bs.As. G.B.A. Zona Sur,Almirante Brown,...,2.0,NaN,NaN,100000.0,USD,NaN,"Casa en Venta en Longchamps, almirante brown ...",SOBRE LA CALLE SAN MARTIN Nº1.025 ENTRE CARLOS...,Casa,Venta
999978,NMZTx5HyF+UGZTe4jeltSg==,Propiedad,2020-05-12,2020-09-15,2020-05-12,-34.650058,-58.425415,Argentina,Capital Federal,Pompeya,...,2.0,NaN,NaN,310000.0,USD,NaN,"Oficina en Venta en Pompeya, Capital federal U...",Sup total del terreno: 199.18 m2\n\na 1 cuadra...,Oficina,Venta
999979,jv4GGNaOj/Z506CJFuiPUw==,Propiedad,2020-05-12,2020-05-20,2020-05-12,-34.602115,-58.405980,Argentina,Capital Federal,Once,...,1.0,180.0,180.0,299001.0,USD,Mensual,Depto.tipo casa de 8 ambientes en Venta en Once,Departamento tipo casa a reciclar primer piso ...,PH,Venta
999980,zLEstdC80JAinfcLGQz5/Q==,Propiedad,2020-05-12,2020-05-20,2020-05-12,-34.415320,-58.645990,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,...,8.0,800.0,800.0,3800000.0,USD,Mensual,Casa en venta al lago central,Imponente casa en la mejor cuadra del barrio E...,Casa,Venta
